<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# Занятие 8. Apache Spark
## Домашнее задание. Анализ веб-логов 

#### Анализ веб-логов - частый случай использования Apache Spark, поскольку логов много, данные часто разреженные. Но с помощью данных из лог-файлов можно улучшать веб-сервисы, рекомендательные системы, обнаружтвать подозрительных пользователей и т.д.

#### План:
#### *Часть 1*: Формат лог-файлов
#### *Часть 2*: Пример анализа лог-файлов
#### *Часть 3*: Анализ серверных лог-файлов
#### *Часть 4*: Исследование запросов с кодом ответа 404

### **Часть 1: Формат лог-файлов**
#### Записи в логах в формате Apache [Common Log Format](http://httpd.apache.org/docs/1.3/logs.html#common) выглядят так:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
#### Подробнее:
* `127.0.0.1`

IP-адрес клиента, сделавшего запрос на сервер.
 
* `-`

Дефисы означают, что личность клиента, сделавшего запрос на сервер, не идентифицирована .
 
* `[01/Aug/1995:00:00:01 -0400]`

Время окончания обработки запроса сервером. Формат:
`[day/month/year:hour:minute:second timezone]`

  * day = 2 цифры
  * month = 3 буквы
  * year = 4 цифры
  * hour = 2 цифры
  * minute = 2 цифры
  * second = 2 цифры
  * zone = (\+ | \-) 4 цифры
 
* `"GET /images/launch-logo.gif HTTP/1.0"`

Первая строка запроса клиента.Тип запроса (`GET`, `POST` и т.д.), [URI](http://en.wikipedia.org/wiki/Uniform_resource_identifier) ресурса, и версия протокола клиента.
 
* `200`

Код статуса, возвращенный сервером. Успешные начинаются с 2, перенаправление запроса - с 3, ошибочные со стороны клиента запросы - с 4, ошибочные со стороны сервера запросы - с 5. 
 
* `1839`

Размер объекта, возвращенного клиенту. Если ничего не было возвращено - это "-" или 0.
 
 
### HTTP серверные веб-логи NASA
#### Используем данные WWW-сервера NASA Kennedy Space Center. [Данные](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html) включают статистику HTTP-запросов за 2 месяца.

### **Предобработка строк лог-файла**
#### Используя регулярные выражения, выберем строки, удовлетворяющие описанному выше шаблону

In [1]:
import re
import datetime

from pyspark.sql import Row

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        return (logline, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = long(0)
    else:
        size = long(match.group(9))
    return (Row(
        host          = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = parse_apache_time(match.group(4)),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = size
    ), 1)

In [2]:
# A regular expression pattern to extract fields from the log line
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'

### **Создание RDD**
#### Функция `parseApacheLogLine` возвращает кортежи с объектом Row и индикатором того, что соответствующая строка попала под шаблон регулярного выражения. 

In [3]:
import sys

logFile = '../../data/NASA_logs'

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc
                   .textFile(logFile)
                   .map(parseApacheLogLine)
                   .cache())

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
                   .cache())

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]))
    failed_logs_count = failed_logs.count()
    if failed_logs_count > 0:
        print('Number of invalid logline: %d' % failed_logs.count())
        for line in failed_logs.take(20):
            print('Invalid logline: %s' % line)

    print('Read %d lines, successfully parsed %d lines, failed to parse %d lines' 
          % (parsed_logs.count(), access_logs.count(), failed_logs.count()))
    return parsed_logs, access_logs, failed_logs


parsed_logs, access_logs, failed_logs = parseLogs()

Number of invalid logline: 108
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:43:39 -0400] "GET / HTTP/1.0 " 200 7131
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:43:57 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0 " 200 5866
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:07 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0 " 200 786
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:11 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0 " 200 363
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:13 -0400] "GET /images/USA-logosmall.gif HTTP/1.0 " 200 234
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:15 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0 " 200 669
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:31 -0400] "GET /shuttle/countdown/ HTTP/1.0 " 200 4673
Invalid logline: ix-sac6-20.ix.netcom.com - - [08/Aug/1995:14:44:41 -0400] "GET /shuttle/missions/sts-69/count69.g

### **Очистка данных**
#### Некоторые строки таким образом не удалось обработать. 
 
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`

Изменим шаблон `APACHE_ACCESS_LOG_PATTERN`, чтобы это исправить.

In [4]:
# This was originally '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'
APACHE_ACCESS_LOG_PATTERN =  '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)'

parsed_logs, access_logs, failed_logs = parseLogs()

Read 1043177 lines, successfully parsed 1043177 lines, failed to parse 0 lines


### **Часть 2: Пример анализа лог-файлов**
 
#### **Статистика объема контента**

In [5]:
# Calculate statistics based on the content size.
content_sizes = access_logs.map(lambda log: log.content_size).cache()
print('Content Size Avg: %i, Min: %i, Max: %s' % (
    content_sizes.reduce(lambda a, b : a + b) / content_sizes.count(),
    content_sizes.min(),
    content_sizes.max()))

Content Size Avg: 17531, Min: 0, Max: 3421948


#### **Анализ кодов ответа сервера**

In [6]:
# Response Code to Count
responseCodeToCount = (access_logs
                       .map(lambda log: (log.response_code, 1))
                       .reduceByKey(lambda a, b : a + b)
                       .cache())
responseCodeToCountList = responseCodeToCount.take(100)
print('Found %d response codes' % len(responseCodeToCountList))
print('Response Code Counts: %s' % responseCodeToCountList)
assert len(responseCodeToCountList) == 7
assert sorted(responseCodeToCountList) == [(200, 940847), (302, 16244), (304, 79824), (403, 58), (404, 6185), (500, 2), (501, 17)]

Found 7 response codes
Response Code Counts: [(200, 940847), (304, 79824), (404, 6185), (500, 2), (501, 17), (302, 16244), (403, 58)]


#### **Отображение статистики кодов ответа с `matplotlib`**
#### Посчитаем доли, приходящиеся на каждый код ответа, и используем Pie-chart.

In [7]:
labels = responseCodeToCount.map(lambda (x, y): x).collect()
print(labels)
count = access_logs.count()
fracs = responseCodeToCount.map(lambda (x, y): (float(y) / count)).collect()
print(fracs)

[200, 304, 404, 500, 501, 302, 403]
[0.9019054292799784, 0.07652009198822443, 0.005929003419362198, 1.9172201841106543e-06, 1.629637156494056e-05, 0.015571662335346735, 5.5599385339208974e-05]


In [8]:
import matplotlib.pyplot as plt


def pie_pct_format(value):
    """ Determine the appropriate format string for the pie chart percentage label
    Args:
        value: value of the pie slice
    Returns:
        str: formated string label; if the slice is too small to fit, returns an empty string for label
    """
    return '' if value < 7 else '%.0f%%' % value

fig = plt.figure(figsize=(4.5, 4.5), facecolor='white', edgecolor='white')
colors = ['yellowgreen', 'lightskyblue', 'gold', 'purple', 'lightcoral', 'yellow', 'black']
explode = (0.05, 0.05, 0.1, 0, 0, 0, 0)
patches, texts, autotexts = plt.pie(fracs, labels=labels, colors=colors,
                                    explode=explode, autopct=pie_pct_format,
                                    shadow=False,  startangle=125)
for text, autotext in zip(texts, autotexts):
    if autotext.get_text() == '':
        text.set_text('')  # If the slice is small to fit, don't show a text label
plt.legend(labels, loc=(0.80, -0.1), shadow=True)

RuntimeError: Invalid DISPLAY variable

#### **Частые адреса**

In [9]:
# Any hosts that has accessed the server more than 10 times.
hostCountPairTuple = access_logs.map(lambda log: (log.host, 1))

hostSum = hostCountPairTuple.reduceByKey(lambda a, b : a + b)

hostMoreThan10 = hostSum.filter(lambda s: s[1] > 10)

hostsPick20 = (hostMoreThan10
               .map(lambda s: s[0])
               .take(20))

print('Any 20 hosts that have accessed more then 10 times: %s' % hostsPick20)
# An example: [u'204.120.34.185', u'204.243.249.9', u'slip1-32.acs.ohio-state.edu', u'lapdog-14.baylor.edu', u'199.77.67.3', u'gs1.cs.ttu.edu', u'haskell.limbex.com', u'alfred.uib.no', u'146.129.66.31', u'manaus.bologna.maraut.it', u'dialup98-110.swipnet.se', u'slip-ppp02.feldspar.com', u'ad03-053.compuserve.com', u'srawlin.opsys.nwa.com', u'199.202.200.52', u'ix-den7-23.ix.netcom.com', u'151.99.247.114', u'w20-575-104.mit.edu', u'205.25.227.20', u'ns.rmc.com']

Any 20 hosts that have accessed more then 10 times: [u'slip3.nilenet.com', u'client-71-31.online.apple.com', u'ix-jac2-16.ix.netcom.com', u'slip124.qlink.queensu.ca', u'rhyolite.geo.umass.edu', u'ix-ftl2-16.ix.netcom.com', u'202.40.17.51', u'dialin14.wantree.com.au', u'y1a.kootenay.net', u'199.242.22.79', u'133.65.48.113', u'weird.stardust.com', u'ucsdtv2.ucsd.edu', u'dialup2.speed.net', u'147.150.5.96', u'152.52.29.20', u'asyn01.lw2.noord.bart.nl', u'bilbo.klautern.fh-rpl.de', u'auto45.bconnex.net', u'ajb.gfdl.gov']


#### **Отображение статистики обращения к ресурсам (endpoints)**

In [10]:
endpoints = (access_logs
             .map(lambda log: (log.endpoint, 1))
             .reduceByKey(lambda a, b : a + b)
             .cache())
ends = endpoints.map(lambda (x, y): x).collect()
counts = endpoints.map(lambda (x, y): y).collect()

fig = plt.figure(figsize=(8,4.2), facecolor='white', edgecolor='white')
plt.axis([0, len(ends), 0, max(counts)])
plt.grid(b=True, which='major', axis='y')
plt.xlabel('Endpoints')
plt.ylabel('Number of Hits')
plt.plot(counts)

RuntimeError: Invalid DISPLAY variable

#### **Частые ресурсы**

In [12]:
# Top Endpoints
endpointCounts = (access_logs
                  .map(lambda log: (log.endpoint, 1))
                  .reduceByKey(lambda a, b : a + b))

topEndpoints = endpointCounts.takeOrdered(10, lambda s: -1 * s[1])

print('Top Ten Endpoints: %s' % topEndpoints)
assert topEndpoints == [(u'/images/NASA-logosmall.gif', 59737), (u'/images/KSC-logosmall.gif', 50452),
                        (u'/images/MOSAIC-logosmall.gif', 43890), (u'/images/USA-logosmall.gif', 43664),
                        (u'/images/WORLD-logosmall.gif', 43277), (u'/images/ksclogo-medium.gif', 41336),
                        (u'/ksc.html', 28582), (u'/history/apollo/images/apollo-logo1.gif', 26778),
                        (u'/images/launch-logo.gif', 24755), (u'/', 20292)]

Top Ten Endpoints: [(u'/images/NASA-logosmall.gif', 59737), (u'/images/KSC-logosmall.gif', 50452), (u'/images/MOSAIC-logosmall.gif', 43890), (u'/images/USA-logosmall.gif', 43664), (u'/images/WORLD-logosmall.gif', 43277), (u'/images/ksclogo-medium.gif', 41336), (u'/ksc.html', 28582), (u'/history/apollo/images/apollo-logo1.gif', 26778), (u'/images/launch-logo.gif', 24755), (u'/', 20292)]


### **Часть 3: Анализ серверных лог-файлов**

#### **10 самых частых ресурсов, обращение к которым вызывало ошибку**
#### Каковы 10 ресурсов (endpoints), при обращении к которым код ответа был не 200? Создайте отсортированный список с 10 самыми частыми ресурсами и числом обращений к ним с кодом ответа, отличным от 200. 

In [ ]:
not200 = access_logs.<Ваш код здесь>

endpointCountPairTuple = not200.<Ваш код здесь>

endpointSum = endpointCountPairTuple.<Ваш код здесь>

topTenErrURLs = endpointSum.<Ваш код здесь>
print('Top Ten failed URLs: %s' % topTenErrURLs)

#### **Число уникальных адресов**
#### Сколько всего уникальных хостов в лог-файле?

In [ ]:
hosts = access_logs.<Ваш код здесь>

uniqueHosts = hosts.<Ваш код здесь>

uniqueHostCount = uniqueHosts.<Ваш код здесь>
print('Unique hosts: %d' % uniqueHostCount)

### Часть 4: Исследование запросов с кодом ответа 404

#### Создайте RDD c запросами лога, имеющими код ответа сервера 404. Закэшируйте RDD `badRecords`.

In [ ]:
badRecords = (access_logs
              <Ваш код здесь>)
print('Found %d 404 URLs' % badRecords.count())

#### Выведите 40 уникальных ресурсов, обращение к которым вызвало статус 404

In [ ]:
badEndpoints = badRecords.<Ваш код здесь>

badUniqueEndpoints = badEndpoints.<Ваш код здесь>

badUniqueEndpointsPick40 = badUniqueEndpoints.take(40)
print('404 URLS: %s' % badUniqueEndpointsPick40)

#### Создайте список топ 20 ресурсов, обращение к которым вызвало статус 404.

In [ ]:
badEndpointsCountPairTuple = badRecords.<Ваш код здесь>

badEndpointsSum = badEndpointsCountPairTuple.<Ваш код здесь>

badEndpointsTop20 = badEndpointsSum.<Ваш код здесь>
print('Top Twenty 404 URLs: %s' % badEndpointsTop20)

#### Создайте список топ 25 адресов, обращение к которым вызвало статус 404.

In [ ]:
errHostsCountPairTuple = badRecords.<Ваш код здесь>

errHostsSum = errHostsCountPairTuple.<Ваш код здесь>

errHostsTop25 = errHostsSum.<Ваш код здесь>
print('Top 25 hosts that generated errors: %s' % errHostsTop25)

#### Посчитайте число запросов со статусом 404 в день 

In [ ]:
errDateCountPairTuple = badRecords.<Ваш код здесь>

errDateSum = errDateCountPairTuple.<Ваш код здесь>

errDateSorted = (errDateSum
                 .sortByKey()).cache()
errByDate = errDateSorted.collect()
print('404 Errors by day: %s' % errByDate)

#### Отображение числа запросов со статусом 404 в день 

In [ ]:
daysWithErrors404 = errDateSorted.<Ваш код здесь>
errors404ByDay = errDateSorted.<Ваш код здесь>

In [ ]:
fig = plt.figure(figsize=(8,4.2), facecolor='white', edgecolor='white')
plt.axis([0, max(daysWithErrors404), 0, max(errors404ByDay)])
plt.grid(b=True, which='major', axis='y')
plt.xlabel('Day')
plt.ylabel('404 Errors')
plt.plot(daysWithErrors404, errors404ByDay)